# Exercise: `mdspan` for Particle Simulation

## Background
The exercise involves converting a basic particle simulation system to use `cuda::std::mdspan`. The original code uses raw pointer arithmetic to handle particle data, which can be error-prone and harder to maintain.

Usage of `cuda::std::mdspan` API for your reference:

```
int height = 2;
int width = 3;
cuda::std::array<int, 6> sd {0, 1, 2, 3, 4, 5};
cuda::std::mdspan md(sd.data(), height, width);

std::printf("md(0, 0) = %d\n", md(0, 0)); // 0
std::printf("md(1, 2) = %d\n", md(1, 2)); // 5

std::printf("size   = %zu\n", md.size());    // 6
std::printf("height = %zu\n", md.extent(0)); // 2
std::printf("width  = %zu\n", md.extent(1)); // 3
``` 

## Exercise
Complete the exercise below by adding the use of `cuda::std::mdspan` instead of using raw pointer arithmetic. The code simulates particles moving in 2D space.

Here's the code to modify:


In [3]:
#Specifying path to where nvcc exists so that the jupyter notebook reads from it. nvcc is the nvidia cuda compiler for executing cuda. 
import os
os.environ['PATH'] = "/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.6.1-cf4xlcbcfpwchqwo5bktxyhjagryzcx6/bin:" + os.environ['PATH']

In [2]:
%%writefile codes/mdspan_particles_exc.cu
#include <thrust/universal_vector.h>
#include <thrust/transform.h>
#include <thrust/execution_policy.h>
#include <cstdio>

void simulate_particles(int num_particles, float dt,
                       const thrust::universal_vector<float> &in,
                             thrust::universal_vector<float> &out) {
    const float *in_ptr = thrust::raw_pointer_cast(in.data());
    
    thrust::transform(
        thrust::device,
        in.begin(), 
        in.end() - num_particles,
        out.begin(),
        [in_ptr, num_particles, dt] __device__ (float val) {
            int idx = &val - in_ptr;
            int particle_idx = idx / 4;
            int component = idx % 4;
            
            if (component < 2) {  // Position components
                return val + dt * in_ptr[particle_idx * 4 + component + 2];
            }
            return val;  // Velocity components unchanged
        }
    );
}

Writing codes/mdspan_particles_exc.cu


In [ ]:
%%bash
nvcc -o codes/mdspan_particles_exc --extended-lambda codes/mdspan_particles_exc.cu
./codes/mdspan_particles_exc

Your task is to:
1. Create an mdspan view of the particle data
2. Replace the raw pointer arithmetic with mdspan operations
3. Use mdspan's extent methods to handle array dimensions

## Hints
<details>
<summary>Click for hints</summary>

- Consider using `cuda::std::mdspan` with dimensions (num_particles, 4) where 4 represents (x, y, vx, vy)
- Use `md(particle_idx, component)` to access particle data instead of pointer arithmetic
- You can use `md.extent(0)` to get number of particles and `md.extent(1)` to get number of components
</details>

## Solution
<details>
<summary>Click to see solution</summary>

```cpp
void simulate_particles(int num_particles, float dt,
                       const thrust::universal_vector<float> &in,
                             thrust::universal_vector<float> &out) {
    
    cuda::std::mdspan particles(thrust::raw_pointer_cast(in.data()), 
                               num_particles, 4);
    
    thrust::transform(
        thrust::device,
        in.begin(), 
        in.end() - particles.extent(0),
        out.begin(),
        [particles, dt] __device__ (float val) {
            int idx = &val - particles.data_handle();
            int particle_idx = idx / particles.extent(1);
            int component = idx % particles.extent(1);
            
            if (component < 2) {  // Position components
                return val + dt * particles(particle_idx, component + 2);
            }
            return val;  // Velocity components unchanged
        }
    );
}
```

Key improvements:
- Created 2D mdspan view of particle data
- Replaced manual pointer arithmetic with mdspan's operator()
- Used mdspan's extent methods for array dimensions
- Code is more readable and less error-prone
</details>